In [ ]:
!pip install plotly pandas numpy scikit-learn matplotlib seaborn matplotlib wandb

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import seaborn as sns
import wandb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [ ]:
!wandb login 85cc7d382dbc235985a5dc344f9d428e2c61df94

In [ ]:
# Initialize W&B
wandb.init(
    project="FrictionAI",
    config={
        "model": "SVR",
        "kernel": "rbf",
        "C_young": 10,
        "epsilon_young": 0.01,
        "C_poisson": 10,
        "epsilon_poisson": 0.01,
        "C_density": 0.9,
        "epsilon_density": 0.1
    }
)

In [ ]:
# Set Seaborn style for better plots
sns.set(style='whitegrid')

In [ ]:
# Data collection
data = {
    'Material_1': [
        'Steel', 'Copper', 'Aluminum', 'Aluminum', 'Brass', 'Cadmium', 'Cadmium',
        'Cast Iron', 'Chromium', 'Copper', 'Copper', 'Copper', 'Glass', 'Glass',
        'Glass', 'Graphite', 'Graphite', 'Nickel', 'Nickel', 'Nylon', 'Plexiglass',
        'Plexiglass', 'Polystyrene', 'Polystyrene', 'Rubber', 'Rubber', 'Brass',
        'Steel', 'Teflon', 'Teflon', 'Wood', 'Wood', 'Wood', 'Wood', 'Zinc', 'Zinc'
    ],
    'E_1': [
        200, 133, 69, 69, 115, 64, 64, 170, 248, 133, 133, 133, 60, 60, 60, 20,
        20, 170, 170, 4, 3.3, 3.3, 2.5, 2.5, 0.01, 0.01, 115, 200, 0.5, 0.5, 10,
        10, 10, 10, 82.7, 82.7
    ],
    'nu_1': [
        0.25, 0.35, 0.33, 0.33, 0.34, 0.31, 0.31, 0.26, 0.31, 0.35, 0.35, 0.35,
        0.25, 0.25, 0.25, 0.2, 0.2, 0.31, 0.31, 0.39, 0.37, 0.37, 0.4, 0.4, 0.47,
        0.47, 0.34, 0.25, 0.47, 0.47, 0.35, 0.35, 0.35, 0.35, 0.25, 0.25
    ],
    'rho_1': [
        8000, 8940, 2700, 2700, 8730, 8650, 8650, 7200, 7190, 8940, 8940, 8940,
        2400, 2400, 2400, 2050, 2050, 8900, 8900, 1130, 1190, 1190, 1040, 1040,
        2300, 2300, 8730, 8000, 2200, 2200, 750, 750, 750, 750, 7120, 7120
    ],
    'Material_2': [
        'Steel', 'Copper', 'Aluminum', 'Steel', 'Cast Iron', 'Cadmium', 'Steel',
        'Cast Iron', 'Chromium', 'Cast Iron', 'Copper', 'Steel', 'Glass', 'Steel',
        'Nickel', 'Graphite', 'Steel', 'Nickel', 'Steel', 'Nylon', 'Plexiglass',
        'Steel', 'Polystyrene', 'Steel', 'Asphalt', 'Concrete', 'Steel', 'Cast Iron',
        'Steel', 'Teflon', 'Wood', 'Copper', 'Steel', 'Concrete', 'Zinc', 'Cast Iron'
    ],
    'E_2': [
        200, 133, 69, 200, 170, 64, 200, 170, 248, 170, 133, 200, 60, 200, 170, 20,
        200, 170, 200, 4, 3.3, 200, 2.5, 200, 3, 17, 200, 170, 200, 0.5, 10, 133,
        200, 17, 82.7, 170
    ],
    'nu_2': [
        0.25, 0.35, 0.33, 0.25, 0.26, 0.31, 0.25, 0.26, 0.31, 0.26, 0.35, 0.25,
        0.25, 0.25, 0.31, 0.2, 0.25, 0.31, 0.25, 0.39, 0.37, 0.25, 0.4, 0.25, 0.35,
        0.17, 0.25, 0.26, 0.25, 0.47, 0.35, 0.35, 0.25, 0.17, 0.25, 0.26
    ],
    'rho_2': [
        8000, 8940, 2700, 8000, 7200, 8650, 8000, 7200, 7190, 7200, 8940, 8000,
        2400, 8000, 8900, 2050, 8000, 8900, 8000, 1130, 1190, 8000, 1040, 8000,
        2500, 2400, 8000, 7200, 8000, 2200, 750, 8940, 8000, 2400, 7120, 7200
    ],
    'mu_static': [
        0.78, 1.21, 1.1, 0.61, 0.4, 0.5, 0.6, 1.1, 0.41, 1.05, 1.0, 0.53, 0.95,
        0.6, 0.78, 0.1, 0.1, 0.8, 0.7, 0.2, 0.8, 0.45, 0.5, 0.3, 0.6, 0.9, 0.35,
        0.4, 0.04, 0.04, 0.3, 0.4, 0.45, 0.62, 0.6, 0.85
    ]
}


data2_incline_plane = {
    "Material_1": [
        "Silver",
        "Silver",
        "Silver",
        "Silver",
        "Aluminium",
        "Aluminium",
        "Gold",
        "Gold",
        "Cadmium",
        "Cadmium",
        "Cobalt",
        "Cobalt",
        "Chromium",
        "Chromium",
        "Copper",
        "Copper",
        "Copper",
        "Copper",
        "Copper",
        "Copper",
        "Iron",
        "Iron",
        "Iron",
        "Iron",
        "Iron",
        "Iron",
        "Iron",
        "Iron",
        "Indium",
        "Manganese",
        "Molybdenum",
        "Molybdenum",
        "Niobium",
        "Nickel",
        "Nickel",
        "Nickel",
        "Lead",
        "Lead",
        "Lead",
        "Lead",
        "Lead",
        "Lead",
        "Platinum",
        "Platinum",
        "Tin",
        "Tin",
        "Titanium",
        "Titanium",
        "Tungsten",
        "Tungsten",
        "Tungsten",
        "Zinc",
        "Zinc",
        "Zinc"
    ],
    "E_1": [
        83.4,   # Silver
        83.4,   # Silver
        83.4,   # Silver
        83.4,   # Silver
        69.0,    # Aluminium
        69.0,    # Aluminium
        79.0,    # Gold
        79.0,    # Gold
        28.0,    # Cadmium
        28.0,    # Cadmium
        211.0,   # Cobalt
        279.0,   # Chromium
        279.0,   # Chromium
        279.0,   # Chromium
        110.0,   # Copper
        110.0,   # Copper
        110.0,   # Copper
        110.0,   # Copper
        110.0,   # Copper
        110.0,   # Copper
        211.0,   # Iron
        211.0,   # Iron
        211.0,   # Iron
        211.0,   # Iron
        211.0,   # Iron
        211.0,   # Iron
        211.0,   # Iron
        211.0,   # Iron
        82.0,    # Indium
        130.0,   # Manganese
        329.0,   # Molybdenum
        329.0,   # Molybdenum
        105.0,   # Niobium
        207.0,   # Nickel
        207.0,   # Nickel
        207.0,   # Nickel
        16.0,    # Lead
        16.0,    # Lead
        16.0,    # Lead
        16.0,    # Lead
        16.0,    # Lead
        16.0,    # Lead
        168.0,   # Platinum
        168.0,   # Platinum
        50.0,    # Tin
        50.0,    # Tin
        116.0,   # Titanium
        116.0,   # Titanium
        411.0,   # Tungsten
        411.0,   # Tungsten
        411.0,   # Tungsten
        108.0,   # Zinc
        108.0,    # Zinc
        108.0    # Zinc
    ],
    "nu_1": [
        0.37,  # Silver
        0.37,  # Silver
        0.37,  # Silver
        0.37,  # Silver
        0.33,  # Aluminium
        0.33,  # Aluminium
        0.42,  # Gold
        0.42,  # Gold
        0.30,  # Cadmium
        0.30,  # Cadmium
        0.31,  # Cobalt
        0.31,  # Cobalt
        0.21,  # Chromium
        0.21,  # Chromium
        0.34,  # Copper
        0.34,  # Copper
        0.34,  # Copper
        0.34,  # Copper
        0.34,  # Copper
        0.34,  # Copper
        0.29,  # Iron
        0.29,  # Iron
        0.29,  # Iron
        0.29,  # Iron
        0.29,  # Iron
        0.29,  # Iron
        0.29,  # Iron
        0.29,  # Iron
        0.45,  # Indium
        0.21,  # Manganese
        0.31,  # Molybdenum
        0.31,  # Molybdenum
        0.40,  # Niobium
        0.31,  # Nickel
        0.31,  # Nickel
        0.31,  # Nickel
        0.44,  # Lead
        0.44,  # Lead
        0.44,  # Lead
        0.44,  # Lead
        0.44,  # Lead
        0.44,  # Lead
        0.38,  # Platinum
        0.38,  # Platinum
        0.36,  # Tin
        0.36,  # Tin
        0.34,  # Titanium
        0.34,  # Titanium
        0.28,  # Tungsten
        0.28,  # Tungsten
        0.28,  # Tungsten
        0.25,  # Zinc
        0.25,   # Zinc
        0.25   # Zinc
    ],
    "rho_1": [
        10490,  # Silver
        10490,  # Silver
        10490,  # Silver
        10490,  # Silver
        2700,   # Aluminium
        2700,   # Aluminium
        19320,  # Gold
        19320,  # Gold
        8650,   # Cadmium
        8650,   # Cadmium
        8900,   # Cobalt
        8900,   # Cobalt
        7190,   # Chromium
        7190,   # Chromium
        8960,   # Copper
        8960,   # Copper
        8960,   # Copper
        8960,   # Copper
        8960,   # Copper
        8960,   # Copper
        7870,   # Iron
        7870,   # Iron
        7870,   # Iron
        7870,   # Iron
        7870,   # Iron
        7870,   # Iron
        7870,   # Iron
        7870,   # Iron
        7310,   # Indium
        7440,   # Manganese
        10220,  # Molybdenum
        10220,  # Molybdenum
        8570,   # Niobium
        8908,   # Nickel
        8908,   # Nickel
        8908,   # Nickel
        11340,  # Lead
        11340,  # Lead
        11340,  # Lead
        11340,  # Lead
        11340,  # Lead
        11340,  # Lead
        21450,  # Platinum
        21450,  # Platinum
        7310,   # Tin
        7310,   # Tin
        4507,   # Titanium
        4507,   # Titanium
        19300,  # Tungsten
        19300,  # Tungsten
        19300,  # Tungsten
        7140,   # Zinc
        7140,    # Zinc
        7140,    # Zinc
    ],
    "Material_2": [
        "Silver",
        "Gold",
        "Copper",
        "Iron",
        "Aluminium",
        "Titanium",
        "Silver",
        "Gold",
        "Cadmium",
        "Iron",
        "Cobalt",
        "Chromium",
        "Cobalt",
        "Chromium",
        "Cobalt",
        "Chromium",
        "Copper",
        "Iron",
        "Nickel",
        "Zinc",
        "Cobalt",
        "Chromium",
        "Iron",
        "Manganese",
        "Molybdenum",
        "Titanium",
        "Tungsten",
        "Zinc",
        "Indium",
        "Manganese",
        "Iron",
        "Molybdenum",
        "Niobium",
        "Chromium",
        "Nickel",
        "Platinum",
        "Silver",
        "Gold",
        "Copper",
        "Chromium",
        "Iron",
        "Lead",
        "Nickel",
        "Platinum",
        "Iron",
        "Tin",
        "Aluminium",
        "Titanium",
        "Copper",
        "Iron",
        "Tungsten",
        "Copper",
        "Iron",
        "Zinc"
    ],
    "E_2": [
        83.0,   # Silver
        79.0,   # Gold
        110.0,  # Copper
        211.0,  # Iron
        69.0,   # Aluminium
        116.0,  # Titanium
        83.0,   # Silver
        79.0,   # Gold
        50.0,   # Cadmium
        211.0,  # Iron
        209.0,  # Cobalt
        279.0,  # Chromium
        209.0,  # Cobalt
        279.0,  # Chromium
        209.0,  # Cobalt
        279.0,  # Chromium
        110.0,  # Copper
        211.0,  # Iron
        207.0,  # Nickel
        108.0,  # Zinc
        209.0,  # Cobalt
        279.0,  # Chromium
        211.0,  # Iron
        200.0,  # Manganese
        329.0,  # Molybdenum
        116.0,  # Titanium
        411.0,  # Tungsten
        108.0,  # Zinc
        11.0,   # Indium
        200.0,  # Manganese
        211.0,  # Iron
        329.0,  # Molybdenum
        105.0,  # Niobium
        279.0,  # Chromium
        207.0,  # Nickel
        168.0,  # Platinum
        83.0,   # Silver
        79.0,   # Gold
        110.0,  # Copper
        279.0,  # Chromium
        211.0,  # Iron
        16.0,   # Lead
        207.0,  # Nickel
        168.0,  # Platinum
        211.0,  # Iron
        50.0,   # Tin
        69.0,   # Aluminium
        116.0,  # Titanium
        110.0,  # Copper
        211.0,  # Iron
        411.0,  # Tungsten
        110.0,  # Copper
        211.0,  # Iron
        108.0   # Zinc
    ],
    "nu_2": [
        0.37,  # Silver
        0.42,  # Gold
        0.34,  # Copper
        0.29,  # Iron
        0.33,  # Aluminium
        0.34,  # Titanium
        0.37,  # Silver
        0.42,  # Gold
        0.30,  # Cadmium
        0.29,  # Iron
        0.31,  # Cobalt
        0.21,  # Chromium
        0.31,  # Cobalt
        0.21,  # Chromium
        0.31,  # Cobalt
        0.21,  # Chromium
        0.34,  # Copper
        0.29,  # Iron
        0.31,  # Nickel
        0.25,  # Zinc
        0.31,  # Cobalt
        0.21,  # Chromium
        0.29,  # Iron
        0.21,  # Manganese
        0.31,  # Molybdenum
        0.34,  # Titanium
        0.28,  # Tungsten
        0.25,  # Zinc
        0.45,  # Indium
        0.21,  # Manganese
        0.29,  # Iron
        0.31,  # Molybdenum
        0.40,  # Niobium
        0.21,  # Chromium
        0.31,  # Nickel
        0.38,  # Platinum
        0.37,  # Silver
        0.42,  # Gold
        0.34,  # Copper
        0.21,  # Chromium
        0.29,  # Iron
        0.44,  # Lead
        0.31,  # Nickel
        0.38,  # Platinum
        0.29,  # Iron
        0.36,  # Tin
        0.33,  # Aluminium
        0.34,  # Titanium
        0.34,  # Copper
        0.29,  # Iron
        0.28,  # Tungsten
        0.34,  # Copper
        0.29,  # Iron
        0.25   # Zinc
    ],
    "rho_2": [
        10490,  # Silver
        19320,  # Gold
        8960,   # Copper
        7870,   # Iron
        2700,   # Aluminium
        4507,   # Titanium
        10490,  # Silver
        19320,  # Gold
        8650,   # Cadmium
        7870,   # Iron
        8900,   # Cobalt
        7190,   # Chromium
        8900,   # Cobalt
        7190,   # Chromium
        8900,   # Cobalt
        7190,   # Chromium
        8960,   # Copper
        7870,   # Iron
        8908,   # Nickel
        7140,   # Zinc
        8900,   # Cobalt
        7190,   # Chromium
        7870,   # Iron
        7440,   # Manganese
        10220,  # Molybdenum
        4507,   # Titanium
        19300,  # Tungsten
        7140,   # Zinc
        7310,   # Indium
        7440,   # Manganese
        7870,   # Iron
        10220,  # Molybdenum
        8570,   # Niobium
        7190,   # Chromium
        8908,   # Nickel
        21450,  # Platinum
        10490,  # Silver
        19320,  # Gold
        8960,   # Copper
        7190,   # Chromium
        7870,   # Iron
        11340,  # Lead
        8908,   # Nickel
        21450,  # Platinum
        7870,   # Iron
        7310,   # Tin
        2700,   # Aluminium
        4507,   # Titanium
        8960,   # Copper
        7870,   # Iron
        19300,  # Tungsten
        8960,   # Copper
        7870,   # Iron
        7140    # Zinc
    ],
    "mu_static": [
        0.5,
        0.53,
        0.48,
        0.49,
        0.57,
        0.54,
        0.53,
        0.49,
        0.79,
        0.52,
        0.56,
        0.41,
        0.41,
        0.46,
        0.44,
        0.46,
        0.55,
        0.50,
        0.49,
        0.56,
        0.41,
        0.48,
        0.51,
        0.51,
        0.46,
        0.49,
        0.47,
        0.55,
        1.46,
        0.69,
        0.46,
        0.44,
        0.46,
        0.59,
        0.50,
        0.64,
        0.73,
        0.61,
        0.55,
        0.53,
        0.54,
        0.90,
        0.64,
        0.55,
        0.55,
        0.74,
        0.54,
        0.55,
        0.41,
        0.47,
        0.51,
        0.56,
        0.55,
        0.75
    ]
}

# Create DataFrames
df_reference = pd.DataFrame(data)
df_incline_plane = pd.DataFrame(data2_incline_plane)

# Combine both datasets
df_combined = pd.concat([df_reference, df_incline_plane], ignore_index=True)

In [ ]:
hyperparams = {
    "C_young": 1000,
    "epsilon_young": 0.001,
    "C_poisson": 1000,
    "epsilon_poisson": 0.001,
    "C_density": 1000,
    "epsilon_density": 0.001,
    "kernel": "rbf",
    "dataset": {
            "reference": {
                "num_samples": len(df_reference['mu_static']),
                "features": ['E_1', 'E_2', 'nu_1', 'nu_2', 'rho_1', 'rho_2'],
                "description": "Reference dataset with material properties and friction coefficients."
            },
            "incline_plane": {
                "num_samples": len(df_incline_plane['mu_static']),
                "features": ['E_1', 'E_2', 'nu_1', 'nu_2', 'rho_1', 'rho_2'],
                "description": "Incline plane dataset with additional material interactions."
            }
        }
}
wandb.config.update(hyperparams)

In [ ]:
# Create a DataFrame
df = pd.DataFrame(data)

# Define features and target
X = df[['E_1', 'nu_1', 'rho_1', 'E_2', 'nu_2', 'rho_2']]
y = df['mu_static']

# Extract individual variables from the DataFrame
density_1 = df['rho_1'].values
density_2 = df['rho_2'].values
young_1 = df['E_1'].values
young_2 = df['E_2'].values
poisson_1 = df['nu_1'].values
poisson_2 = df['nu_2'].values
mu_static = df['mu_static'].values

# Feature Engineering
df['E_diff'] = df['E_1'] - df['E_2']
df['nu_diff'] = df['nu_1'] - df['nu_2']
df['rho_diff'] = df['rho_1'] - df['rho_2']
df['E_ratio'] = df['E_1'] / df['E_2']
df['nu_ratio'] = df['nu_1'] / df['nu_2']
df['rho_ratio'] = df['rho_1'] / df['rho_2']

# Interaction between materials type
df['E_nu_interaction_1'] = df['E_1'] * df['nu_1']
df['E_rho_interaction_1'] = df['E_1'] * df['rho_1']
df['nu_rho_interaction_1'] = df['nu_1'] * df['rho_1']

df['E_nu_interaction_2'] = df['E_2'] * df['nu_2']
df['E_rho_interaction_2'] = df['E_2'] * df['rho_2']
df['nu_rho_interaction_2'] = df['nu_2'] * df['rho_2']


In [ ]:
# Prepare feature matrices for SVR
X_young = np.vstack((young_1, young_2)).T
X_poisson = np.vstack((poisson_1, poisson_2)).T
X_density = np.vstack((density_1, density_2)).T

In [ ]:
# Standardize the features
scaler_young = StandardScaler()
X_young_scaled = scaler_young.fit_transform(X_young)

scaler_poisson = StandardScaler()
X_poisson_scaled = scaler_poisson.fit_transform(X_poisson)

scaler_density = StandardScaler()
X_density_scaled = scaler_density.fit_transform(X_density)

In [ ]:
# Train SVR models
svr_young = SVR(kernel=hyperparams['kernel'], C=hyperparams['C_young'], epsilon=hyperparams['epsilon_young'])
svr_young.fit(X_young_scaled, mu_static)

svr_poisson = SVR(kernel=hyperparams['kernel'], C=hyperparams['C_poisson'], epsilon=hyperparams['epsilon_poisson'])
svr_poisson.fit(X_poisson_scaled, mu_static)

svr_density = SVR(kernel=hyperparams['kernel'], C=hyperparams['C_density'], epsilon=hyperparams['epsilon_density'])
svr_density.fit(X_density_scaled, mu_static)

In [ ]:
# Predictions on training data
y_pred_young = svr_young.predict(X_young_scaled)
y_pred_poisson = svr_poisson.predict(X_poisson_scaled)
y_pred_density = svr_density.predict(X_density_scaled)

# Compute error metrics for svr_young
rmse_young = np.sqrt(mean_squared_error(mu_static, y_pred_young))
mae_young = mean_absolute_error(mu_static, y_pred_young)
r2_young = r2_score(mu_static, y_pred_young)

# Compute error metrics for svr_poisson
rmse_poisson = np.sqrt(mean_squared_error(mu_static, y_pred_poisson))
mae_poisson = mean_absolute_error(mu_static, y_pred_poisson)
r2_poisson = r2_score(mu_static, y_pred_poisson)

# Compute error metrics for svr_density
rmse_density = np.sqrt(mean_squared_error(mu_static, y_pred_density))
mae_density = mean_absolute_error(mu_static, y_pred_density)
r2_density = r2_score(mu_static, y_pred_density)

# Print the results
wandb.log({
    "Young's Modulus": {
        "RMSE": rmse_young,
        "MAE": mae_young,
        "R2_Score": r2_young
    }
})

wandb.log({
    "Poisson's Ratio": {
        "RMSE": rmse_poisson,
        "MAE": mae_poisson,
        "R2_Score": r2_poisson
    }
})

wandb.log({
    "Density": {
        "RMSE": rmse_density,
        "MAE": mae_density,
        "R2_Score": r2_density
    }
})

In [ ]:
# Grid for Young's Modulus
x_grid_young = np.linspace(min(young_1), max(young_1), 1000)
y_grid_young = np.linspace(min(young_2), max(young_2), 1000)
X_grid_young, Y_grid_young = np.meshgrid(x_grid_young, y_grid_young)

grid_points_young = np.vstack([X_grid_young.ravel(), Y_grid_young.ravel()]).T
grid_points_young_scaled = scaler_young.transform(grid_points_young)
Z_pred_young = svr_young.predict(grid_points_young_scaled).reshape(X_grid_young.shape)

In [ ]:
# Grid for Poisson Ratios
x_grid_poisson = np.linspace(min(poisson_1), max(poisson_1), 1000)
y_grid_poisson = np.linspace(min(poisson_2), max(poisson_2), 1000)
X_grid_poisson, Y_grid_poisson = np.meshgrid(x_grid_poisson, y_grid_poisson)

grid_points_poisson = np.vstack([X_grid_poisson.ravel(), Y_grid_poisson.ravel()]).T
grid_points_poisson_scaled = scaler_poisson.transform(grid_points_poisson)
Z_pred_poisson = svr_poisson.predict(grid_points_poisson_scaled).reshape(X_grid_poisson.shape)

In [ ]:
# Grid for Densities
x_grid_density1 = np.linspace(min(density_1), max(density_1), 1000)
y_grid_density2 = np.linspace(min(density_2), max(density_2), 1000)
X_grid_density1, Y_grid_density2 = np.meshgrid(x_grid_density1, y_grid_density2)

grid_points_density = np.vstack([X_grid_density1.ravel(), Y_grid_density2.ravel()]).T
grid_points_density_scaled = scaler_density.transform(grid_points_density)
Z_pred_density = svr_density.predict(grid_points_density_scaled).reshape(X_grid_density1.shape)

In [ ]:
# Plot 1: Interactive 3D Plot of Young's Modulus
fig1 = go.Figure(data=[
    go.Surface(
        x=X_grid_young,
        y=Y_grid_young,
        z=Z_pred_young,
        colorscale='Viridis',
        opacity=0.7,
        name='SVR Prediction Surface'
    ),
    go.Scatter3d(
        x=young_1,
        y=young_2,
        z=mu_static,
        mode='markers',
        marker=dict(size=5, color='red'),
        name='Real Data Points'
    )
])
fig1.update_layout(
    title="Plot of Friction Coefficient with Young's Modulus (SVR Prediction vs Real)",
    scene=dict(
        xaxis_title="Young's Modulus (GPa)",
        yaxis_title="Young's Modulus (GPa)",
        zaxis_title='Coefficient of Friction'
    )
)

fig1.show()

# Send to wandb
wandb.log({"Young's Modulus Plot": fig1})

In [ ]:
# Plot 2: Interactive 3D Plot of Poisson Ratio
fig2 = go.Figure()

# Add the surface plot
fig2.add_trace(go.Surface(
    x=X_grid_poisson,
    y=Y_grid_poisson,
    z=Z_pred_poisson,
    colorscale='Viridis',
    opacity=0.7,
    name='SVR Prediction Surface'
))

# Add the scatter plot for real data points
fig2.add_trace(go.Scatter3d(
    x=poisson_1,
    y=poisson_2,
    z=mu_static,
    mode='markers',
    marker=dict(
        size=5,
        color='blue',
        symbol='circle'
    ),
    name='Real Data Points'
))

# Update the layout
fig2.update_layout(
    title="Plot of Friction Coefficient with Poisson Ratios (SVR Prediction vs Real)",
    scene=dict(
        xaxis_title='Poisson Ratio 1',
        yaxis_title='Poisson Ratio 2',
        zaxis_title='Coefficient of Friction'
    ),
    legend=dict(
        x=0.8,
        y=0.9
    )
)

# Show the figure
fig2.show()
wandb.log({"Plot of Friction Coefficient with Poisson Ratios": fig2})

In [ ]:
# Plot 3: Interactive 3D Plot of Densities
fig3 = go.Figure()

# Add the surface plot
fig3.add_trace(go.Surface(
    x=X_grid_density1,
    y=Y_grid_density2,
    z=Z_pred_density,
    colorscale='Viridis',
    opacity=0.7,
    name='SVR Prediction Surface'
))

# Add the scatter plot for real data points
fig3.add_trace(go.Scatter3d(
    x=density_1,
    y=density_2,
    z=mu_static,
    mode='markers',
    marker=dict(
        size=5,
        color='green',
        symbol='circle'
    ),
    name='Real Data Points'
))

# Update the layout
fig3.update_layout(
    title="Plot of Friction Coefficient with Densities (SVR Prediction vs Real)",
    scene=dict(
        xaxis_title='Density 1 (kg/m³)',
        yaxis_title='Density 2 (kg/m³)',
        zaxis_title='Coefficient of Friction'
    ),
    legend=dict(
        x=0.8,
        y=0.9
    )
)

# Show the figure
fig3.show()
wandb.log({"Plot of Friction Coefficient with Densities": fig3})

In [ ]:
df2 = pd.DataFrame(data2_incline_plane)

young_1_new = df2['E_1'].values
young_2_new = df2['E_2'].values
poisson_1_new = df2['nu_1'].values
poisson_2_new = df2['nu_2'].values
density_1_new = df2['rho_1'].values
density_2_new = df2['rho_2'].values


# Prepare feature matrices for prediction
X_young_new = np.vstack((young_1_new, young_2_new)).T
X_poisson_new = np.vstack((poisson_1_new, poisson_2_new)).T
X_density_new = np.vstack((density_1_new, density_2_new)).T

# Standardize the new features using the same scalers
X_young_new_scaled = scaler_young.transform(X_young_new)
X_poisson_new_scaled = scaler_poisson.transform(X_poisson_new)
X_density_new_scaled = scaler_density.transform(X_density_new)

# Make predictions using the trained models
mu_pred_young = svr_young.predict(X_young_new_scaled)
mu_pred_poisson = svr_poisson.predict(X_poisson_new_scaled)
mu_pred_density = svr_density.predict(X_density_new_scaled)

# Print the predictions
for idx in range(len(df2)):
    material = df2['Material_1'][idx]
    actual_mu = df2['mu_static'][idx]

    pred_young = mu_pred_young[idx]
    pred_poisson = mu_pred_poisson[idx]
    pred_density = mu_pred_density[idx]

    wandb.log({
        "Material": material,
        "Young's Modulus Prediction": pred_young,
        "Poisson's Ratio Prediction": pred_poisson,
        "Density Prediction": pred_density,
        "Actual Coefficient of Friction": actual_mu,
        "Error Young's Modulus": abs(pred_young - actual_mu),
        "Error Poisson's Ratio": abs(pred_poisson - actual_mu),
        "Error Density": abs(pred_density - actual_mu)
    })


In [ ]:
x_grid_young_new = np.linspace(min(young_1_new), max(young_1_new), 1000)
y_grid_young_new = np.linspace(min(young_2_new), max(young_2_new), 1000)
X_grid_young_new, Y_grid_young_new = np.meshgrid(x_grid_young_new, y_grid_young_new)

grid_points_young_new = np.vstack([X_grid_young_new.ravel(), Y_grid_young_new.ravel()]).T
grid_points_young_scaled_new = scaler_young.transform(grid_points_young_new)
Z_pred_young_new = svr_young.predict(grid_points_young_scaled_new).reshape(X_grid_young_new.shape)


fig1.add_trace(go.Scatter3d(
    x=young_1_new,
    y=young_2_new,
    z=df2['mu_static'].values,
    mode='markers',
    marker=dict(size=5, color='orange', symbol='x'),
    name="New Data Points"
))

fig1.add_trace(go.Surface(
    x=X_grid_young_new,
    y=Y_grid_young_new,
    z=Z_pred_young_new,
    colorscale='Plasma',
    opacity=0.5,
    name='SVR Prediction Surface (High Resolution)',
    showscale=False
))

fig1.show()
wandb.log({"Plot of Friction Coefficient with Young with Tests": fig1})

In [ ]:
x_grid_poisson_new = np.linspace(min(poisson_1_new), max(poisson_1_new), 1000)
y_grid_poisson_new = np.linspace(min(poisson_2_new), max(poisson_2_new), 1000)
X_grid_poisson_new, Y_grid_poisson_new = np.meshgrid(x_grid_poisson_new, y_grid_poisson_new)

grid_points_poisson_new = np.vstack([X_grid_poisson_new.ravel(), Y_grid_poisson_new.ravel()]).T
grid_points_poisson_scaled_new = scaler_poisson.transform(grid_points_poisson_new)
Z_pred_poisson_new = svr_poisson.predict(grid_points_poisson_scaled_new).reshape(X_grid_poisson_new.shape)

fig2.add_trace(go.Scatter3d(
    x=poisson_1_new,
    y=poisson_2_new,
    z=df2['mu_static'].values,
    mode='markers',
    marker=dict(size=5, color='orange', symbol='x'),
    name="New Data Points"
))

fig2.add_trace(go.Surface(
    x=X_grid_poisson_new,
    y=Y_grid_poisson_new,
    z=Z_pred_poisson_new,
    colorscale='Plasma',
    opacity=0.5,
    name='SVR Prediction Surface (High Resolution)',
    showscale=False
))

fig2.show()
wandb.log({"Plot of Friction Coefficient with Poisson with Tests": fig2})

In [ ]:
x_grid_density_new = np.linspace(min(density_1_new), max(density_1_new), 1000)
y_grid_density_new = np.linspace(min(density_2_new), max(density_2_new), 1000)
X_grid_density_new, Y_grid_density_new = np.meshgrid(x_grid_density_new, y_grid_density_new)

grid_points_density_new = np.vstack([X_grid_density_new.ravel(), Y_grid_density_new.ravel()]).T
grid_points_density_scaled_new = scaler_density.transform(grid_points_density_new)
Z_pred_density_new = svr_density.predict(grid_points_density_scaled_new).reshape(X_grid_density_new.shape)


fig3.add_trace(go.Scatter3d(
    x=density_1_new,
    y=density_2_new,
    z=df2['mu_static'].values,
    mode='markers',
    marker=dict(size=5, color='orange', symbol='x'),
    name="New Data Points"
))

fig3.add_trace(go.Surface(
    x=X_grid_density_new,
    y=Y_grid_density_new,
    z=Z_pred_density_new,
    colorscale='Plasma',
    opacity=0.5,
    name='SVR Prediction Surface (High Resolution)',
    showscale=False
))

fig3.show()
wandb.log({"Plot of Friction Coefficient with Densities with Tests": fig3})

In [ ]:
wandb.finish()